In [36]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier as xgb
from sklearn.metrics import accuracy_score
from scipy.special import boxcox1p
from scipy.stats import skew
from scipy.stats import boxcox_normmax
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_seq_items = 100
pd.options.display.max_rows = 100

## 将字符转化为日期
def to_date(x):
    if len(str(x))>7 and int(str(x)[:4])>1900 and int(str(x)[:4])<2019:
        x = str(x)
        date = x[:4]+"-"+x[4:6]+"-"+x[6:8]
        return date
    else:
        return np.NaN
def judge(x):
    if len(str(x))>3:
        return int(str(x).split(" ")[0])
    else:
        return np.NaN
    
path = './xby/'
file = ['call_data.csv','dpi_data.csv', 'prd_data.csv','trmnl_data.csv', 'train_result.csv']
data = pd.read_csv(path + 'cust_data.csv')
for csv in file:
    tmp = pd.read_csv(path + csv)
    data = pd.merge(data, tmp, left_on="user", right_on="user", how='left')
raw_data = data.copy()

###
print("数据读取完毕，开始数据处理....")
answer = {
     '0F2E4CC10EDBE80F':1500,
     '56AFA2A526F96CC9':3075,
     '7C26FADD409BD4B9':68000,
     '816A9BEBED2D7C99':6132,
     'C7E2941B65C6CCD6':700}
ans = pd.DataFrame(answer, index=[1]).T
ans.columns = ['ans']
ans['rate'] = ans['ans'] / ans['ans'].sum()
ans

数据读取完毕，开始数据处理....


,ans,rate
0F2E4CC10EDBE80F,1500,0.018890
56AFA2A526F96CC9,3075,0.038725
7C26FADD409BD4B9,68000,0.856348
816A9BEBED2D7C99,6132,0.077222
C7E2941B65C6CCD6,700,0.008815


In [37]:
## 终端型号、类型、产品编码字段处理：
# data.drop(['term_model','pro_brand', 'product_nbr'], axis=1, inplace=True)
object_filed = ['product_nbr', 'pro_brand', 'term_model']
for filed in object_filed:
    df1 = pd.DataFrame(data[filed].value_counts()).reset_index()
    df1.columns = [filed, "counts"]
    df1 = df1[df1["counts"]>2000].reset_index()
    df1 = df1.drop(["counts"], axis=1)
    new_name = "new_"+filed
    df1.columns = [new_name, filed]
    data = pd.merge(data, df1, left_on=filed, right_on=filed, how='left')
    data = data.drop(filed, axis=1)

## 对通话次数进行分箱，对用户进行分组：
dt_m_list = ['dt_m_1000','dt_m_1003','dt_m_1004','dt_m_1005','dt_m_1006','dt_m_1009','dt_m_1011','dt_m_1012','dt_m_1015','dt_m_1017',
             'dt_m_1027','dt_m_1028','dt_m_1032','dt_m_1034','dt_m_1035','dt_m_1041','dt_m_1043','dt_m_1051','dt_m_1052','dt_m_1067',
             'dt_m_1068','dt_m_1073','dt_m_1074','dt_m_1075','dt_m_1085','dt_m_1086','dt_m_1087','dt_m_1096','dt_m_1099','dt_m_1102',
             'dt_m_1105','dt_m_1108','dt_m_1111','dt_m_1594','dt_m_1601','dt_m_1617','dt_m_1618','dt_m_1620','dt_m_1630','dt_m_1633']
for filed in dt_m_list:
    data[filed][(data[filed]<0) | (data[filed].isna())] = 0
bins,labels = [-np.inf, 500, 1000, 2000, 5000, np.inf],[0, 1, 2, 3, 4]
data['dt_m_2000'] = pd.cut(data['dt_m_1000'], bins, labels=labels).values.tolist()   ##通话总次数分箱

## 自注册短信的发送时间间隔
tmp = pd.DataFrame(pd.to_datetime("2019-12-26")-pd.to_datetime(data['register_date']))
data["register_date"] = tmp['register_date'].apply(lambda x: judge(x))
data['register_date'] = data.groupby('dt_m_2000')['register_date'].transform(lambda x: x.fillna(x.mean()))


## 开通日期、入网时长、生日字段计算时间间隔
date_filed = ['open_date', 'cust_access_net_dt', 'birth_date']
for filed in date_filed:
    data[filed] = data[filed].apply(lambda x: to_date(x))
    tmp = pd.DataFrame(pd.to_datetime("2019-12-26")-pd.to_datetime(data[filed]))
    data[filed] = tmp[filed].apply(lambda x: judge(x))

## 客户的入网时长单位转化为月：并添加两个字段入网时长为负数、空值的设置为0，入网时长大于240个月的设置为1，其余设置为2，并对入网时长分箱
data['cust_access_net_dt'] = data['cust_access_net_dt'].apply(lambda x: np.round(x/30))
data['new_cust_access_net_dt_1'] = data[['cust_access_net_dt']].copy()
data['new_cust_access_net_dt_1'][(data['cust_access_net_dt']<0)| (data['cust_access_net_dt'].isna())] = 0
data['new_cust_access_net_dt_1'][data['cust_access_net_dt']>240] = 1
data['new_cust_access_net_dt_1'][(data['new_cust_access_net_dt_1']!=0) & (data['new_cust_access_net_dt_1']!=1)] = 2
data['cust_access_net_dt'] = data.groupby('dt_m_2000')['cust_access_net_dt'].transform(lambda x: x.fillna(x.mean()))


## 客户的信用等级：添加一个字段将值为-1设置为0，值为999999设置为1，空值设置为3， 其他值设置为2
data['new_credit_level'] = data[['credit_level']].copy()
data['new_credit_level'][data['credit_level']==-1] = 0
data['new_credit_level'][data['credit_level']==999999] = 1
data['new_credit_level'][data['credit_level'].isna()] = 3
data['new_credit_level'][(data['new_credit_level']!=0) & (data['new_credit_level']!=1) & (data['new_credit_level']!=3)] = 2
data.drop(['credit_level'], axis=1, inplace=True)

## 会员级别：添加一个字段将值为9999的设置为0其余值设置为1：
data['new_membership_level'] = data[['membership_level']].copy()
data['new_membership_level'][data['membership_level']==9999] = 0
data['new_membership_level'][data['membership_level'].isna()] = 2
data['new_membership_level'][(data['new_membership_level']!=0) & (data['new_membership_level']!=2)] = 1
data.drop(['membership_level'], axis=1, inplace=True)

## 客户性别：添加一个字段将客户性别为-1的设置为0，其余值设置为1，空值设置为2：
data['new_gender'] = data[['gender']].copy()
data['new_gender'][data['gender']==-1] = 0
data['new_gender'][data['gender'].isna()] = 2
data['new_gender'][(data['new_gender']!=0) & (data['new_gender']!=2)] = 1
data.drop(['gender'], axis=1, inplace=True)

## 终端价值：
data['new_market_price'] = data[['market_price']].copy()
data['new_market_price'][data['market_price'].isna()] = 0
data['new_market_price'][~data['market_price'].isna()] = 1
data['market_price'] = data.groupby('dt_m_2000')['market_price'].transform(lambda x: x.fillna(x.mean()))
    
    
## 客户生日：添加两个字段，计算客户年龄，对年龄大于90或小于5或为空的设置为0其余值设置为1，将年龄为空值的用非空值的年龄均值填充并将年龄分箱统计
data['birth_date'] = data['birth_date'].apply(lambda x: np.round(x/365))
data['new_birth_date_1'] = data[['birth_date']].copy()
data['new_birth_date_1'][(data['birth_date']<5) | (data['birth_date']>90)] = 0
data['new_birth_date_1'][(data['new_birth_date_1']>=5) & (data['birth_date']<=90)] = 1
data['new_birth_date_1'][data['birth_date'].isna()] = 2
data['birth_date'][data['new_birth_date_1']==2] = np.NaN
data['birth_date'] = data.groupby('dt_m_2000')['birth_date'].transform(lambda x: x.fillna(x.mean()))

## 客户积分：
data['new_cust_point_1'] = data[['cust_point']].copy()
data['new_cust_point_1'][data['new_cust_point_1']<0]  = 0
data['new_cust_point_1'][data['new_cust_point_1']==0] = 1
data['new_cust_point_1'][data['new_cust_point_1']>0] = 2
data['new_cust_point_1'][data['new_cust_point_1'].isna()] = 3
data['cust_point'][data['cust_point']<0] = np.NaN
data['cust_point'] = data.groupby('dt_m_2000')['cust_point'].transform(lambda x: x.fillna(x.mean()))

## 客户的星级级别：
data['star_level'][data['star_level'].isna()] = 99999
data['inet_pd_inst_cnt'] = data.groupby('dt_m_2000')['inet_pd_inst_cnt'].transform(lambda x: x.fillna(x.mean()))

## 开户日期
data['new_open_date_1'] = data[['open_date']].copy()
data['new_open_date_1'][data['new_open_date_1'].isna()] = 0
data['new_open_date_1'][data['new_open_date_1']!=0]  = 1
data['open_date'] = data.groupby('dt_m_2000')['open_date'].transform(lambda x: x.fillna(x.mean()))

## 客户访问APP1的次数：
data['new_app1_visits_1'] = data[['app1_visits']].copy()
data['new_app1_visits_1'][data['new_app1_visits_1'].isna()] = 0
data['new_app1_visits_1'][data['new_app1_visits_1']!=0]  = 1
### APP使用次数衍生
app_filed = ['app1_visits','app2_visits','app3_visits','app4_visits','app5_visits','app6_visits','app7_visits','app8_visits']
for filed in app_filed:
    data[filed] = data.groupby('dt_m_2000')[filed].transform(lambda x: x.fillna(x.mean()))
app_visits_min = data[app_filed].min(axis=1)
app_visits_max = data[app_filed].max(axis=1)
app_visits_sum = data[app_filed].sum(axis=1)
##  使用APP次数最多和最少比值
data['app_visits_rate1'] = app_visits_max / (app_visits_min+1)
data['app_visits_rate2'] = app_visits_max / (app_visits_sum+1)
data['app_visits_rate3'] = app_visits_min / (app_visits_sum+1)
## 本月以来是否访问过这个8个APP
data['app_visits_is'] = app_visits_sum
data['app_visits_is'][data['app_visits_is']>10] = 1     
data['app_visits_is'][data['app_visits_is']<11] = 0
### 使用APP次数最多、最少的APP名称
tmp = data[app_filed]
tmp.columns = [i for i in range(1,9)]
data['app_visits_idxman'] = tmp.idxmax(axis=1)
data['app_visits_idxmin'] = tmp.idxmin(axis=1)

## 通话数据处理：
data['dt_m_2001']=data['dt_m_1006']/(data['dt_m_1000']+1)  ##主叫占总次数之比                  
data['dt_m_2002']=data['dt_m_1012']/(data['dt_m_1000']+1)  ##被叫占总次数之比                  
data['dt_m_2003']=data['dt_m_1009']/(data['dt_m_1003']+1)  ##主叫时长占总时长之比              
data['dt_m_2004']=data['dt_m_1015']/(data['dt_m_1003']+1)  ##被叫时长占总时长之比              
data['dt_m_2005']=data['dt_m_1006']-(data['dt_m_1012'])    ##被叫次数和主叫次数之差            
data['dt_m_2006']=data['dt_m_1009']-(data['dt_m_1015'])    ##被叫时长和主叫时长之差            
data['dt_m_2007']=data['dt_m_1011']-(data['dt_m_1017'])    ##被叫最大时长和主叫最大时长之差    
data['dt_m_2008']=data['dt_m_1009']/(data['dt_m_1006']+1)  ##主叫平均时长                      
data['dt_m_2009']=data['dt_m_1015']/(data['dt_m_1012']+1)  ##被叫平均时长                      
data['dt_m_2010']=data['dt_m_1000']/(data['dt_m_1041']+1)  ##通话次数和通话人数比值            
data['dt_m_2011']=data['dt_m_1041']-(data['dt_m_1051'])    ##被叫通话人数                      
data['dt_m_2012']=data['dt_m_1051']/(data['dt_m_1006']+1)  ##主叫通话次数和主叫通话人数比值    
data['dt_m_2013']=data['dt_m_2011']/(data['dt_m_1012']+1)  ##被叫通话次数和主叫通话人数比值    
data['dt_m_2014']=data['dt_m_1096']/(data['dt_m_1000']+1)  ##长途通话次数占总次数比值      
data['dt_m_2015']=data['dt_m_1099']/(data['dt_m_1003']+1)  ##长途通话时长占总时长比值          
data['dt_m_2016']=data['dt_m_1102']/(data['dt_m_1006']+1)  ##长途主叫通话次数占主叫总次数比值  
data['dt_m_2017']=data['dt_m_1108']/(data['dt_m_1012']+1)  ##长途被叫通话次数占被叫总次数比值  
data['dt_m_2018']=data['dt_m_1105']/(data['dt_m_1009']+1)  ##长途主叫通话时长占主叫总时长比值  
data['dt_m_2019']=data['dt_m_1111']/(data['dt_m_1015']+1)  ##长途被叫通话时长占被叫总时长比值  
data['dt_m_2021']=data['dt_m_1052']/(data['dt_m_1043']+1)  ##近三个月主叫通话人数占总人数比值  
data['dt_m_2022']=data['dt_m_1052']-(data['dt_m_1043'])    ##近三个月被叫通话人数       
data['dt_m_2023']=data['dt_m_2022']/(data['dt_m_1052']+1)  ##近三个月主被叫之比          
data['dt_m_2024']=data['dt_m_1006']/(data['dt_m_1012']+1)  ##主被叫通话次数之比       
data['dt_m_2025']=data['dt_m_1009']/(data['dt_m_1015']+1)  ##主被叫通话时长之比       
data['dt_m_2026']=data['dt_m_1630']/(data['dt_m_1633']+1)  ##主被发送短信数之比       
data['dt_m_2027']=data['dt_m_1618']/(data['dt_m_1003']+1)  ##上网时长和通话时长之比      
data['dt_m_2028']=data['dt_m_1102']/(data['dt_m_1108']+1)  ##长途通话次数主被叫之比   
data['dt_m_2029']=data['dt_m_1105']/(data['dt_m_1111']+1)  ##长途通话时长主被叫之比   
data['dt_m_2030']=data['dt_m_1073']/(data['dt_m_1000']+1)  ##07-18点通话次数占总次数之比  
data['dt_m_2031']=data['dt_m_1074']/(data['dt_m_1000']+1)  ##18-22点通话次数占总次数之比  
data['dt_m_2032']=data['dt_m_1075']/(data['dt_m_1000']+1)  ##22-07点通话次数占总次数之比  
data['dt_m_2035']=data['dt_m_1630']/(data['dt_m_1620']+1)  ##平均每秒使用流量     
data['dt_m_2020']=1-data['dt_m_1035']                      ##1min以上主叫占比
data['dt_m_2033']=data['dt_m_1043']/3                      ##近三个月每月平均通话人数  
data['dt_m_2034']=data['dt_m_1052']/3                      ##近三个月每月平均主叫通话人数
data['dt_m_2036']=data['dt_m_1041']-(data['dt_m_2033'])    ##平均通话人数和本月通话人数之差 
data['dt_m_2037']=data['dt_m_1051']-(data['dt_m_2034'])    ##平均主叫人数和本月主叫人数之差 
data['dt_m_2038']=data['dt_m_1041']/(data['dt_m_2033']+1)  ##平均通话人数和本月通话人数之比
data['dt_m_2039']=data['dt_m_1051']/(data['dt_m_2034']+1)  ##平均主叫人数和本月主叫人数之比 
data['dt_m_2040']=data['dt_m_2001']/(data['dt_m_2002'])    ##主叫占总次数比值和被叫占总次数之比之差
data['dt_m_2041']=data['dt_m_1085']*data['dt_m_1006']      ##07-18点主叫通话次数
data['dt_m_2042']=data['dt_m_1073']-data['dt_m_2041']      ##07-18点被叫通话次数
data['dt_m_2043']=data['dt_m_2041']/(data['dt_m_2042']+1)  ##07-18点主被叫次数之比
data['dt_m_2044']=data['dt_m_1032']*data['dt_m_1006']      ##10秒内的主叫次数
data['dt_m_2045']=data['dt_m_2044']/data['dt_m_1006']      ##10秒内主叫次数占主叫总次数的之比
data['dt_m_2046']=data['dt_m_1034']*data['dt_m_1006']      ##30秒内的主叫次数
data['dt_m_2047']=data['dt_m_2046']/data['dt_m_1006']      ##30秒内主叫次数占主叫总次数的之比
data['dt_m_2048']=data['dt_m_1035']*data['dt_m_1006']      ##60秒内的主叫次数
data['dt_m_2049']=data['dt_m_2048']/data['dt_m_1006']      ##60秒内主叫次数占主叫总次数的之比
data['dt_m_2050']=data['dt_m_1067']/(data['dt_m_1068']+1)  ##本月主叫对端号码个数占近三个月对端号码个数之比
data['dt_m_2051']=data['dt_m_1073']/(data['dt_m_1074']+1)  ##本月7-18点工作时段通话次数与18-22点晚间时段通话次数之比
data['dt_m_2052']=data['dt_m_1073']/(data['dt_m_1075']+1)  ##本月7-18点工作时段通话次数与22-07点晚间时段通话次数之比
data['dt_m_2053']=data['dt_m_1074']/(data['dt_m_1075']+1)  ##本月18-22点工作时段通话次数与22-07点晚间时段通话次数之比
data['dt_m_2054']=data['dt_m_1006']/(data['dt_m_1051']+1)  ##本月平均每人通话次数
data['dt_m_2055']=data['dt_m_1009']/(data['dt_m_1051']+1)  ##本月平均每人通话时长
data['dt_m_2056']=data['dt_m_1105']/(data['dt_m_1102']+1)  ##本月平均每次主叫长途通话时长
data['dt_m_2057']=data['dt_m_1111']/(data['dt_m_1108']+1)  ##本月平均每次主叫长途通话时长
data['dt_m_2058']=data['dt_m_1003']/(data['dt_m_1041']+1)  ##本月平均每次通话时长
data['dt_m_2059']=data['dt_m_2058']/(data['dt_m_2008']+1)  ##本月平均每次通话时长和主叫平均时长之比
data['dt_m_2060']=data['dt_m_2058']-(data['dt_m_2008'])    ##本月平均每次通话时长和主叫平均时长之差


# ### 将类别字段的值类型转化为objective：
# object_list = ["user","label","new_cust_access_net_dt_1","new_credit_level","new_membership_level","new_gender","new_market_price",
#                "new_birth_date_1","new_cust_point_1","star_level","new_open_date_1","new_app1_visits_1","app_visits_idxman",
#                "app_visits_idxmin"]
# data[object_list] = data[object_list].astype("object")

# print("开始进行数据变换......")
# ## 将长尾分布的字段进行数值变换：
# numeric = ['dt_m_2005','dt_m_2006','dt_m_2007','dt_m_2008','dt_m_2009','dt_m_2011','dt_m_2022','dt_m_2035','dt_m_2033','dt_m_2034',
#             'dt_m_2036','dt_m_2037','dt_m_2041','dt_m_2042','dt_m_2044','dt_m_2046','dt_m_1000','dt_m_1003','dt_m_1004','dt_m_1005',
#             'dt_m_1006','dt_m_1009','dt_m_1011','dt_m_1012','dt_m_1015','dt_m_1017','dt_m_1027','dt_m_1028','dt_m_1041','dt_m_1043',
#             'dt_m_1051','dt_m_1052','dt_m_1067','dt_m_1068','dt_m_1073','dt_m_1074','dt_m_1075','dt_m_1096','dt_m_1099','dt_m_1102',
#             'dt_m_1105','dt_m_1108','dt_m_1111','dt_m_1594','dt_m_1601','dt_m_1617','dt_m_1618','dt_m_1620','dt_m_1630','dt_m_1633',
#             'app1_visits','app2_visits','app3_visits','app4_visits','app5_visits','app6_visits','app7_visits','app8_visits',
#             'app_visits_diff','open_date']
# skew_features = data[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)
# high_skew = skew_features[skew_features > 0.5]
# ### 对偏度大的字段做boxcoxlp转换：
# for filed in high_skew.index:
#     data[filed] = data[filed].apply(lambda x: np.abs(x))
#     data[filed] = boxcox1p(data[filed], boxcox_normmax(data[filed] + 1))
    
##### 含有标签数据集的大小：
train = data[~data['label'].isnull()]
test = data[data['label'].isnull()].drop(['label'], axis=1)
print("含有标签的数据集大小：", train.shape, "没有标签的数据集大小：", test.shape)

含有标签的数据集大小： (322757, 138) 没有标签的数据集大小： (79407, 137)


In [38]:
####  数据集的划分：
# df = data.sample(frac=0.005)
df = data.copy()
tmp1 = df[df['label']=='7C26FADD409BD4B9']
tmp2 = tmp1.sample(frac=0.35).reset_index(drop=True)
tmp3 = df[raw_data['label']!='7C26FADD409BD4B9'].reset_index(drop=True)
df = pd.concat([tmp2, tmp3], axis=0).reset_index(drop=True)
tmp = pd.DataFrame(data['label'].value_counts())
tmp.columns = ['train']
tmp['train_rate'] = tmp['train'] / tmp['train'].sum()
answer = {
     '0F2E4CC10EDBE80F':1500,
     '56AFA2A526F96CC9':3075,
     '7C26FADD409BD4B9':68000,
     '816A9BEBED2D7C99':6132,
     'C7E2941B65C6CCD6':700}
ans = pd.DataFrame(answer, index=[1]).T
ans.columns = ['ans']
ans['ans_rate'] = ans['ans'] / ans['ans'].sum()
mubiao = pd.concat([tmp, ans], axis=1).sort_values(by=['train'])
################################################################################################
all_data,target = df.drop(['label', 'user'], axis=1), df[['label']]
# object_list = ["new_cust_access_net_dt_1","new_credit_level","new_membership_level","new_gender","new_market_price","new_birth_date_1",
#                "new_cust_point_1","star_level","new_open_date_1","new_app1_visits_1","app_visits_idxman","app_visits_idxmin"]
# all_data[object_list] = all_data[object_list].astype("object")
# all_data = pd.get_dummies(all_data).reset_index(drop=True)
train_data,test_data,target=all_data[~target['label'].isnull()],all_data[target['label'].isnull()],target[~target['label'].isnull()]['label']
cat = pd.Categorical(target,categories=target.unique(),ordered=True)
new_target = cat.codes
train_x, test_x, train_y, test_y = train_test_split(train_data, new_target, test_size=0.15, shuffle = True, random_state=0)
X_train,y_train = train_x.values,train_y
X_test,y_test = test_x.values,test_y
print("训练数据集大小：", X_train.shape, "测试数据集大小：", X_test.shape)
mubiao

训练数据集大小： (118037, 136) 测试数据集大小： (20831, 136)


,train,train_rate,ans,ans_rate
C7E2941B65C6CCD6,2293,0.007104,700,0.008815
56AFA2A526F96CC9,8954,0.027742,3075,0.038725
0F2E4CC10EDBE80F,10792,0.033437,1500,0.018890
816A9BEBED2D7C99,17812,0.055187,6132,0.077222
7C26FADD409BD4B9,282906,0.876529,68000,0.856348


In [41]:
params = {  
    'boosting_type': 'gbdt', 
    'max_depth': 7,
    'num_leaves': 35, 
    'min_data_in_leaf': 100,  
    'bagging_fraction': 0.8,
    'feature_fraction': 0.6, 
    'bagging_freq': 2, 
    'objective': 'multiclass',  
    'num_class': 5,  
    'metric': 'multi_error',  
    'learning_rate': 0.02,  
    'min_gain_to_split': 0.01,  
    'lambda_l1': 0.0,  
    'lambda_l2': 1.0,  
    'verbose': -1
}  
print('Training...')
trn_data = lgb.Dataset(X_train, y_train)
val_data = lgb.Dataset(X_test, y_test)
clf = lgb.train(params, 
                trn_data, 
                num_boost_round = 5000,
                valid_sets = [trn_data,val_data], 
                verbose_eval = 200,
                early_stopping_rounds=200)
print('Predicting...')
y_prob = clf.predict(X_test, num_iteration=clf.best_iteration)
y_pred = [list(x).index(max(x)) for x in y_prob]
f1 = metrics.f1_score(y_pred, test_y,average="micro")
print("f1 score: {:<8.5f}".format(f1))

###
y_prob = clf.predict(test_data, num_iteration=clf.best_iteration)
y_pred = [list(x).index(max(x)) for x in y_prob]
result = pd.DataFrame(data[data['label'].isnull()]['user'])
result['pred_label'] = [cat.categories[i] for i in y_pred]
result.to_csv(path + "result_v3.0.csv", header=None, index=False)
result['pred_label'].value_counts()

Training...
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_error: 0.0422664	valid_1's multi_error: 0.0460852
[400]	training's multi_error: 0.0368274	valid_1's multi_error: 0.0437329
[600]	training's multi_error: 0.0327948	valid_1's multi_error: 0.0431088
[800]	training's multi_error: 0.0288045	valid_1's multi_error: 0.0421967
[1000]	training's multi_error: 0.0258224	valid_1's multi_error: 0.0419087
Early stopping, best iteration is:
[941]	training's multi_error: 0.0266527	valid_1's multi_error: 0.0418127
Predicting...
f1 score: 0.95819 


7C26FADD409BD4B9    70624
816A9BEBED2D7C99     4568
56AFA2A526F96CC9     2360
0F2E4CC10EDBE80F     1128
C7E2941B65C6CCD6      727
Name: pred_label, dtype: int64

In [56]:
clf.feature_importance()

array([1957, 2347, 2019, 1070, 1049, 1382,  702,  625, 1656,  910,  774,
       1540, 1134,  839, 1249,  920, 1497, 2446, 1635, 1672,  862, 1121,
       1078, 1515, 1890, 2477,  774, 1332,  875,  937, 1245, 1302,  773,
        867,  752,  751,  281,  360, 1047,  463, 1768, 1567, 1485, 1242,
       1153, 2494, 1241,  472, 1281, 1385, 1369, 1131, 1399, 2973,  245,
       1753, 1522, 4442,  902, 1225,    1,  248, 1091,   23,  412,   22,
        498,    4,  974,  219, 1385, 1795,  195,    0,  118,  265,  638,
        782,  907,  755,  843, 1018, 1457, 1457, 1763, 1650,  953, 1831,
       1876, 1184, 1119, 1529,  458, 1483,  486, 1585, 1526, 1815,  595,
        689, 1328, 1312, 1142, 1310,  931, 1137,  920, 1446, 1080,  432,
        606, 1431, 1223, 2027, 2061,  621,  835, 1207, 1086, 1486, 1115,
       1218,  699, 1037,  619, 1801, 1022, 1243, 1589, 1689, 1399, 1435,
        662, 1198, 1332, 1107])

In [51]:
### 输出混淆矩阵和分类报告：
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_prob = clf.predict(X_test, num_iteration=clf.best_iteration)
y_pred = [list(x).index(max(x)) for x in y_prob]
f1 = metrics.f1_score(y_pred, test_y,average="micro")
print("f1 score: {:<8.5f}".format(f1))
print("分类报告：\n",classification_report(y_pred, y_test))
print("混淆矩阵：\n",confusion_matrix(y_pred, y_test))

f1 score: 0.95819 
分类报告：
               precision    recall  f1-score   support

           0       0.99      0.97      0.98     15112
           1       0.93      0.89      0.91      2863
           2       0.90      0.89      0.90       347
           3       0.92      0.95      0.94      1212
           4       0.75      0.94      0.83      1297

   micro avg       0.96      0.96      0.96     20831
   macro avg       0.90      0.93      0.91     20831
weighted avg       0.96      0.96      0.96     20831

混淆矩阵：
 [[14720   103    28    38   223]
 [   61  2561     2    52   187]
 [   26     3   308    10     0]
 [   26    25     3  1156     2]
 [   19    59     0     4  1215]]


In [42]:
jj = pd.read_csv("./xby/11.csv", header=None)
jj.columns = ['user', 'label']
rr = pd.merge(jj, result, left_on='user', right_on='user', how='left')
rr[rr['label'] == rr['pred_label']]['label'].value_counts(), sum(rr['label'] == rr['pred_label'])/len(rr) #0.9826589595375722

(7C26FADD409BD4B9    70135
 816A9BEBED2D7C99     4215
 56AFA2A526F96CC9     2130
 0F2E4CC10EDBE80F      935
 C7E2941B65C6CCD6      637
 Name: label, dtype: int64, 0.9829360131978289)

Training...
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_error: 0.0447233	valid_1's multi_error: 0.0473333
[400]	training's multi_error: 0.0403856	valid_1's multi_error: 0.044453
[600]	training's multi_error: 0.0372171	valid_1's multi_error: 0.0438769
[800]	training's multi_error: 0.0343791	valid_1's multi_error: 0.0434449
[1000]	training's multi_error: 0.0320662	valid_1's multi_error: 0.0429648
[1200]	training's multi_error: 0.0298805	valid_1's multi_error: 0.0423407
[1400]	training's multi_error: 0.0280421	valid_1's multi_error: 0.0423887
[1600]	training's multi_error: 0.0259919	valid_1's multi_error: 0.0420047
[1800]	training's multi_error: 0.0241365	valid_1's multi_error: 0.0421487
Early stopping, best iteration is:
[1600]	training's multi_error: 0.0259919	valid_1's multi_error: 0.0420047
Predicting...
f1 score: 0.95800 
7C26FADD409BD4B9    70636
816A9BEBED2D7C99     4579
56AFA2A526F96CC9     2358
0F2E4CC10EDBE80F     1122
C7E2941B65C6CCD6      712
Name: pred_label, dtype: int64
#########################################################################################################
Training...
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_error: 0.0235983	valid_1's multi_error: 0.0506421
[400]	training's multi_error: 0.00702202	valid_1's multi_error: 0.0497377
[600]	training's multi_error: 0.000425577	valid_1's multi_error: 0.0490143
[800]	training's multi_error: 0	valid_1's multi_error: 0.048954
Early stopping, best iteration is:
[731]	training's multi_error: 0	valid_1's multi_error: 0.048954
Predicting...
f1 score: 0.95105 
7C26FADD409BD4B9    70408
816A9BEBED2D7C99     4654
56AFA2A526F96CC9     2378
0F2E4CC10EDBE80F     1166
C7E2941B65C6CCD6      801
Name: pred_label, dtype: int64
#########################################################################################################
Training...
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_error: 0.040733	valid_1's multi_error: 0.0478614
[400]	training's multi_error: 0.0329219	valid_1's multi_error: 0.0436849
[600]	training's multi_error: 0.0276269	valid_1's multi_error: 0.0428208
[800]	training's multi_error: 0.0234249	valid_1's multi_error: 0.0417647
[1000]	training's multi_error: 0.0201886	valid_1's multi_error: 0.0408526
[1200]	training's multi_error: 0.0170709	valid_1's multi_error: 0.0407566
Early stopping, best iteration is:
[1110]	training's multi_error: 0.0186636	valid_1's multi_error: 0.0405645
Predicting...
f1 score: 0.95944 
7C26FADD409BD4B9    70707
816A9BEBED2D7C99     4524
56AFA2A526F96CC9     2371
0F2E4CC10EDBE80F     1099
C7E2941B65C6CCD6      706
Name: pred_label, dtype: int64
#########################################################################################################
Training...
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_error: 0.0227452	valid_1's multi_error: 0.0239807
[400]	training's multi_error: 0.0199385	valid_1's multi_error: 0.0219771
[600]	training's multi_error: 0.0182363	valid_1's multi_error: 0.0215227
[800]	training's multi_error: 0.0169532	valid_1's multi_error: 0.0213161
[1000]	training's multi_error: 0.0158634	valid_1's multi_error: 0.0212748
[1200]	training's multi_error: 0.01485	valid_1's multi_error: 0.0210063
[1400]	training's multi_error: 0.0139096	valid_1's multi_error: 0.020903
[1600]	training's multi_error: 0.0130931	valid_1's multi_error: 0.0207791
[1800]	training's multi_error: 0.012262	valid_1's multi_error: 0.0207378
Early stopping, best iteration is:
[1607]	training's multi_error: 0.0130785	valid_1's multi_error: 0.0207171
Predicting...
f1 score: 0.97928 
7C26FADD409BD4B9    71655
816A9BEBED2D7C99     4168
56AFA2A526F96CC9     2162
0F2E4CC10EDBE80F      929
C7E2941B65C6CCD6      493
Name: pred_label, dtype: int64
#########################################################################################################
Training...
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_error: 0.0227452	valid_1's multi_error: 0.0239807
[400]	training's multi_error: 0.0199385	valid_1's multi_error: 0.0219771
[600]	training's multi_error: 0.0182363	valid_1's multi_error: 0.0215227
[800]	training's multi_error: 0.0169532	valid_1's multi_error: 0.0213161
[1000]	training's multi_error: 0.0158634	valid_1's multi_error: 0.0212748
[1200]	training's multi_error: 0.01485	valid_1's multi_error: 0.0210063
[1400]	training's multi_error: 0.0139096	valid_1's multi_error: 0.020903
[1600]	training's multi_error: 0.0130931	valid_1's multi_error: 0.0207791
[1800]	training's multi_error: 0.012262	valid_1's multi_error: 0.0207378
Early stopping, best iteration is:
[1607]	training's multi_error: 0.0130785	valid_1's multi_error: 0.0207171
Predicting...
f1 score: 0.97928 
7C26FADD409BD4B9    71655
816A9BEBED2D7C99     4168
56AFA2A526F96CC9     2162
0F2E4CC10EDBE80F      929
C7E2941B65C6CCD6      493
Name: pred_label, dtype: int64
#########################################################################################################
Training...
Training until validation scores don't improve for 200 rounds.
[200]	training's multi_error: 0.0225723	valid_1's multi_error: 0.0249473
[400]	training's multi_error: 0.0198168	valid_1's multi_error: 0.0228405
[600]	training's multi_error: 0.0180074	valid_1's multi_error: 0.0223324
[800]	training's multi_error: 0.0167061	valid_1's multi_error: 0.0219606
[1000]	training's multi_error: 0.0154007	valid_1's multi_error: 0.0217871
[1200]	training's multi_error: 0.0142894	valid_1's multi_error: 0.0217003
[1400]	training's multi_error: 0.0132319	valid_1's multi_error: 0.0215764
[1600]	training's multi_error: 0.0123767	valid_1's multi_error: 0.0214525
[1800]	training's multi_error: 0.0113935	valid_1's multi_error: 0.0214897
[2000]	training's multi_error: 0.0105425	valid_1's multi_error: 0.0214401
Early stopping, best iteration is:
[1906]	training's multi_error: 0.010935	valid_1's multi_error: 0.0213533
Predicting...
f1 score: 0.97865 
7C26FADD409BD4B9    71684
816A9BEBED2D7C99     4128
56AFA2A526F96CC9     2146
0F2E4CC10EDBE80F      952
C7E2941B65C6CCD6      497
Name: pred_label, dtype: int64

In [20]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
##############

train = lgb.Dataset(X_train, y_train)
valid = lgb.Dataset(X_test, y_test, reference=train)
parameters = {
              'max_depth': [5, 8, 10, 12, 14, 16, 18],
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15, 0.2],
              'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_freq': [2, 4, 5, 6, 8],
              'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
              'lambda_l2': [0, 10, 15, 35, 40],
              'cat_smooth': [1, 10, 15, 20, 35]
}
gbm = lgb.LGBMClassifier(boosting_type='gbdt',
                         objective = 'multiclass',
                         metric = 'multi_error',
                         verbose = 0,
                         average="micro",
                         learning_rate = 0.01,
                         num_leaves = 35,
                         num_class=5,
                         feature_fraction=0.8,
                         bagging_fraction= 0.9,
                         bagging_freq= 8,
                         lambda_l1= 0.6,
                         lambda_l2= 0)
# 有了gridsearch我们便不需要fit函数
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='accuracy', cv=3)
gsearch.fit(X_train, y_train)

KeyboardInterrupt: 

In [187]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=7, min_samples_leaf=200, min_samples_split=200, n_estimators=200, n_jobs=-1)
print(clf)
clf.fit(train_x, train_y)

y_pred = clf.predict(X_test)
f1 = metrics.f1_score(y_pred, test_y,average="micro")
print("f1 score: {:<8.5f}".format(f1))

###
rf_pred = clf.predict(test_data)
result = pd.DataFrame(data[data['label'].isnull()]['user'])
result['pred_label'] = [cat.categories[i] for i in rf_pred]
result['pred_label'].value_counts()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=200,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
f1 score: 0.96358 


7C26FADD409BD4B9    72810
816A9BEBED2D7C99     3836
56AFA2A526F96CC9     2024
0F2E4CC10EDBE80F      699
C7E2941B65C6CCD6       38
Name: pred_label, dtype: int64

In [58]:
feature_importance = pd.DataFrame({'name':train_data.columns, 'value':clf.feature_importance()})
feature_importance['per'] = 100 * feature_importance['value'] / feature_importance['value'].sum()
feature_importance.sort_values(by=['value'], ascending=False).head(20)

,name,value,per
57,new_product_nbr,4442,2.818367
53,open_date,2973,1.886314
45,app1_visits,2494,1.582397
25,dt_m_1068,2477,1.571611
17,dt_m_1032,2446,1.551942
1,birth_date,2347,1.489128
114,dt_m_2039,2061,1.307666
113,dt_m_2038,2027,1.286094
2,cust_point,2019,1.281018
0,cust_access_net_dt,1957,1.241680


In [41]:
# feature_importance = pd.DataFrame({'name':train_data.columns, 'value':clf.feature_importances_})
# feature_importance['per'] = 100 * feature_importance['value'] / feature_importance['value'].sum()
# feature_importance.sort_values(by=['value'], ascending=False).head(20)

In [46]:
# from catboost import CatBoostRegressor, CatBoostClassifier
# all_data,target = data.drop(['label', 'user'], axis=1), data[['label']]
# train_data,test_data,target=all_data[~target['label'].isnull()],all_data[target['label'].isnull()],target[~target['label'].isnull()]['label']
# cat = pd.Categorical(target,categories=target.unique(),ordered=True)
# new_target = cat.codes
# x_train,x_test,y_train,y_test = train_test_split(train_data,new_target,test_size=0.25,random_state=0)
# print("训练集大小：",x_train.shape, "验证集大小：",x_test.shape, "预测集大小：",test_data.shape)

# ##############
# cat_attrs =  ["last_year_capture_user_flag","new_register_date_1","new_cust_access_net_dt_1",
#               "new_cust_access_net_dt_2","star_level","new_credit_level",
#               "new_membership_level","new_gender","new_birth_date_1","new_birth_date_2","new_cust_point_1",
#               "new_cust_point_2","new_open_date_1","new_open_date_2","new_app1_visits_1","new_app1_visits_2","new_app3_visits_2",
#               "new_app4_visits_2","new_app5_visits_2","app_visits_idxmin","app_visits_idxman","new_dt_m_1000"]
# catboost_model = CatBoostClassifier(
#     iterations=2000,
#     od_type='Iter',
#     od_wait=120,
#     max_depth=8,
#     learning_rate=0.02,
#     l2_leaf_reg=9,
#     random_seed=2019,
#     metric_period=50,
#     fold_len_multiplier=1.1,
#     loss_function='MultiClass',
#     logging_level='Verbose')
 
# catboost_model.fit(x_train, y_train)#, cat_features=cat_attrs)
# y_pred = catboost_model.predict(x_test)
# # 模型评价
# f1 = f1_score(y_test, y_pred, average='macro')
# print('f1 : ', f1)

# # ###
# y_pred=catboost_model.predict(test_data)
# pd.DataFrame(y_pred)[0].value_counts()

In [47]:
# feature_importance = pd.DataFrame({'name':train_data.columns, 'value':catboost_model.feature_importances_})
# feature_importance.sort_values(by=['value'], ascending=False).head(40)

In [30]:
import torch
from torch import nn

############################################
tmp_data = data.drop(['label', 'user'], axis=1)
dtypes = tmp_data.dtypes

# ############################################
# object_filed = ["new_cust_access_net_dt_1","new_credit_level","new_membership_level","new_gender","new_market_price","new_birth_date_1",
#                  "new_cust_point_1","star_level","new_open_date_1","new_app1_visits_1","new_dt_m_1000","last_year_capture_user_flag"]
# for filed in object_filed:
#     tmp_data[filed] = tmp_data[filed].astype("object")
# float_filed = dtypes[dtypes!='object'].index.tolist()
# tmp_data[float_filed] = tmp_data[float_filed].apply(lambda x: (x - x.mean()) / (x.std()))
# tmp_data = pd.get_dummies(tmp_data, dummy_na=True)

############################################
all_data,target = tmp_data, data[['label']]
train_data,test_data,target=all_data[~target['label'].isnull()],all_data[target['label'].isnull()],target[~target['label'].isnull()]['label']
cat = pd.Categorical(target,categories=target.unique(),ordered=True)
new_target = cat.codes

def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    np.random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i:min(i+batch_size, num_examples)])
        yield features.index_select(0, j), labels.index_select(0, j)
batch_size = 128
features = torch.from_numpy(train_data.values).float()
labels = torch.from_numpy(new_target).long()

## 继承nn.Module类哎构造模型
## 这里定义的MLP类重载了nn.Module类的__init__函数和forward函数，他们分别用于创建模型参数和定义前向传播算法
class MLP(nn.Module):
    def __init__(self, n):
        super(MLP, self).__init__()
        self.hiddent1 = nn.Linear(n, 512)
#         self.hiddent2 = nn.Linear(512, 256)
        self.act = nn.ReLU()
#         self.hiddent3 = nn.Linear(256, 128)
        self.hiddent4 = nn.Linear(512, 5)
        
    ## 定义模型的前向传播算法，即如何根据输入x计算返回所需要的模型输出    
    def forward(self, x):
        x = self.act(self.hiddent1(x))
#         x = self.act(self.hiddent2(x))
#         x = self.act(self.hiddent3(x))
        x = self.act(self.hiddent4(x))
        return x
model = MLP(train_data.shape[1])
model

MLP(
  (hiddent1): Linear(in_features=72, out_features=512, bias=True)
  (act): ReLU()
  (hiddent4): Linear(in_features=512, out_features=5, bias=True)
)

In [31]:
from torch.nn import init
#define the initial function to init the layer's parameters for the network
def weigth_init(m):
    if isinstance(m, nn.Conv2d):
        init.xavier_uniform_(m.weight.data)
        init.constant_(m.bias.data,0.1)
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
    elif isinstance(m, nn.Linear):
        m.weight.data.normal_(0,0.01)
        m.bias.data.zero_()

lr = 0.01
num_epochs = 50
model = MLP(train_data.shape[1])
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
model.apply(weigth_init)

for epoch in range(num_epochs):  # 训练模型一共需要num_epochs个迭代周期
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。X
    # 和y分别是小批量样本的特征和标签
    for X, y in data_iter(batch_size, features, labels):
        pred = model(X)
        l = loss(pred, y).sum()  # l是有关小批量X和y的损失
        optimizer.zero_grad()  # 对模型参数做一个优化，并且将梯度清0
        l.backward()  # 计算梯度
        print("loss: ", l.data)

        ## 参数更新
        optimizer.step()
    train_l = loss(model(features), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

...

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tensor(nan)
loss:  tenso

KeyboardInterrupt: 

In [110]:
### 导入模块
import numpy as np
import pandas as pd
import lightgbm as lgb
# from xgboost import XGBClassifier as xgb
from sklearn import metrics


all_data,target = data.drop(['label', 'user'], axis=1), data[['label']]
train_data,test_data,target=all_data[~target['label'].isnull()],all_data[target['label'].isnull()],target[~target['label'].isnull()]['label']
cat = pd.Categorical(target,categories=target.unique(),ordered=True)
new_target = cat.codes
train_x, test_x, train_y, test_y = train_test_split(train_data, new_target, shuffle = True, random_state = 2019)
### 数据转换
print('数据转换')
lgb_train = lgb.Dataset(train_x, train_y, free_raw_data=False)
lgb_eval = lgb.Dataset(test_x, test_y, reference=lgb_train,free_raw_data=False)

数据转换


In [144]:
### 设置初始参数--不含交叉验证参数
print('设置参数')
params = {
          'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'num_class': 5,  
          'metric': 'multi_error', 
          }

### 交叉验证(调参)
print('交叉验证')
min_merror = float('Inf')
best_params = {}

# # 准确率
# print("调参1：提高准确率")
# for num_leaves in range(20,200,5):
#     for max_depth in range(3,8,1):
#         params['num_leaves'] = num_leaves
#         params['max_depth'] = max_depth

#         cv_results = lgb.cv(
#                             params,
#                             lgb_train,
#                             seed=2018,
#                             nfold=3,
#                             early_stopping_rounds=10,
#                             verbose_eval=True
#                             )
            
#         mean_merror = pd.Series(cv_results['multi_error-mean']).min()
#         boost_rounds = pd.Series(cv_results['multi_error-mean']).argmin()
            
#         if mean_merror < min_merror:
#             min_merror = mean_merror
#             best_params['num_leaves'] = num_leaves
#             best_params['max_depth'] = max_depth
            
# params['num_leaves'] = best_params['num_leaves']
# params['max_depth'] = best_params['max_depth']

# 过拟合
print("调参2：降低过拟合")
for max_bin in range(2,255,5):
    for min_data_in_leaf in range(10,200,5):
            params['max_bin'] = max_bin
            params['min_data_in_leaf'] = min_data_in_leaf
            
                    
            mean_merror = pd.Series(cv_results['multi_error-mean']).min()
            boost_rounds = pd.Series(cv_results['multi_error-mean']).argmin()

            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['max_bin']= max_bin
                best_params['min_data_in_leaf'] = min_data_in_leaf

params['min_data_in_leaf'] = best_params['min_data_in_leaf']
params['max_bin'] = best_params['max_bin']

print("调参3：降低过拟合")
for feature_fraction in [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    for bagging_fraction in [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=42,
                                nfold=3,
                                early_stopping_rounds=3,
                                verbose_eval=True
                                )
                    
            mean_merror = pd.Series(cv_results['multi_error-mean']).min()
            boost_rounds = pd.Series(cv_results['multi_error-mean']).argmin()

            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq

params['feature_fraction'] = best_params['feature_fraction']
params['bagging_fraction'] = best_params['bagging_fraction']
params['bagging_freq'] = best_params['bagging_freq']

print("调参4：降低过拟合")
for lambda_l1 in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    for lambda_l2 in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            params['lambda_l1'] = lambda_l1
            params['lambda_l2'] = lambda_l2
            params['min_split_gain'] = min_split_gain
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=42,
                                nfold=3,
                                early_stopping_rounds=3,
                                verbose_eval=True
                                )
                    
            mean_merror = pd.Series(cv_results['multi_error-mean']).min()
            boost_rounds = pd.Series(cv_results['multi_error-mean']).argmin()

            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['lambda_l1'] = lambda_l1
                best_params['lambda_l2'] = lambda_l2
                best_params['min_split_gain'] = min_split_gain

params['lambda_l1'] = best_params['lambda_l1']
params['lambda_l2'] = best_params['lambda_l2']
params['min_split_gain'] = best_params['min_split_gain']


print(best_params)

### 训练
params['learning_rate']=0.01
lgb.train(
          params,                     # 参数字典
          lgb_train,                  # 训练集
          valid_sets=lgb_eval,        # 验证集
          num_boost_round=2000,       # 迭代次数
          early_stopping_rounds=50    # 早停次数
          )

设置参数
交叉验证
调参2：降低过拟合
调参3：降低过拟合
[1]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[2]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[3]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[4]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[1]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[2]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[3]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[4]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[1]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[2]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[3]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[4]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[1]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[2]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[3]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[4]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[1]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[2]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[3]	cv_agg's multi_error: 0.123482 + 1.30118e-05
[4]	cv_agg's multi_error: 0.123482 + 1.

KeyboardInterrupt: 

In [126]:
### 训练
params['learning_rate']=0.01
clf = lgb.train(
          params,                     # 参数字典
          lgb_train,                  # 训练集
          valid_sets=lgb_eval,        # 验证集
          num_boost_round=2000,       # 迭代次数
          early_stopping_rounds=50    # 早停次数
          )

[1]	valid_0's multi_error: 0.123435
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_error: 0.123435
[3]	valid_0's multi_error: 0.123435
[4]	valid_0's multi_error: 0.123435
[5]	valid_0's multi_error: 0.123435
[6]	valid_0's multi_error: 0.123435
[7]	valid_0's multi_error: 0.123435
[8]	valid_0's multi_error: 0.123435
[9]	valid_0's multi_error: 0.123435
[10]	valid_0's multi_error: 0.123435
[11]	valid_0's multi_error: 0.123435
[12]	valid_0's multi_error: 0.123435
[13]	valid_0's multi_error: 0.123435
[14]	valid_0's multi_error: 0.123435
[15]	valid_0's multi_error: 0.123435
[16]	valid_0's multi_error: 0.123435
[17]	valid_0's multi_error: 0.123435
[18]	valid_0's multi_error: 0.123435
[19]	valid_0's multi_error: 0.123435
[20]	valid_0's multi_error: 0.123435
[21]	valid_0's multi_error: 0.123435
[22]	valid_0's multi_error: 0.123435
[23]	valid_0's multi_error: 0.123435
[24]	valid_0's multi_error: 0.123435
[25]	valid_0's multi_error: 0.123435
[26]	valid_0's multi_e

In [129]:
y_prob = clf.predict(test_data, num_iteration=clf.best_iteration)
y_pred = [list(x).index(max(x)) for x in y_prob]
result = pd.DataFrame(data[data['label'].isnull()]['user'])
result['pred_label'] = [cat.categories[i] for i in y_pred]
result['pred_label'].value_counts()

7C26FADD409BD4B9    79407
Name: pred_label, dtype: int64

In [145]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV


all_data,target = data.drop(['label', 'user'], axis=1), data[['label']]
train_data,test_data,target=all_data[~target['label'].isnull()],all_data[target['label'].isnull()],target[~target['label'].isnull()]['label']
cat = pd.Categorical(target,categories=target.unique(),ordered=True)
new_target = cat.codes
x_train, x_test, y_train, y_test = train_test_split(train_data, new_target, shuffle = True, random_state = 2019)

In [153]:
### 建立模型
print('# 建立模型')
estimator = LGBMClassifier(
                     num_class=5,
                     objective='multiclass',            # 二分类问题
                     learning_rate=0.01,             # 默认0.1，学习率
                     subsample_for_bin=1  ,         #
                     subsample=1,                   # 默认1，
                     metric = 'multi_error',            # 评估指标
                     silent =True,                  # 输出中间过程
                     reg_alpha=0.0,                 # L1正则化系数
                     min_split_gain=0.0             # 默认0，分裂最小权重
                     )

### 网格搜素
print('# 网格搜索')
param_grid = {
    'max_depth':list(range(3,10,1)),
    'max_bin':list(range(1, 255, 5)),
    'min_data_in_leaf':list(range(10, 200, 5)),
    'feature_fraction':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'bagging_fraction':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'lambda_l1':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'lambda_l2':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'min_split_gain':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'n_estimators':list(range(50,200,50)),
    'num_leaves':list(range(25,80,5)),
    'min_child_weight':list(range(2,10,1))
        
        }

gs = GridSearchCV(estimator,              # 分类器
                  param_grid,             # 参数字典
                  scoring='neg_log_loss', # 评价标准
                  cv=3,                   # 三折交叉验证
                  verbose = 2,            # 打印全部中间过程（0/1/2）
                  n_jobs=-1)               # 并行计算CPU个数

gs.fit(x_train,y_train)

# 建立模型
# 网格搜索
Fitting 3 folds for each of 1848 candidates, totalling 5544 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.3min


KeyboardInterrupt: 

In [154]:
import xgboost as xgb                              # xgboost模型
from xgboost.sklearn import XGBClassifier          # xgboost模型（sklearn）
# from sklearn import cross_validation               # 交叉验证
from sklearn import metrics                        # 性能度量
from sklearn.model_selection import GridSearchCV   # 网格搜索

In [ ]:
### 训练模型（交叉验证）
"""
# alg                    算法
# dtrain                 训练集
# dtest                  测试集 
# predictors  
# useTrainCV 
# cv_flods 
# early_stopping_rounds 

# XGBClassifier是封装在sklearn中的xgboost模型
"""
def modelfit(alg, dtrain, dtest, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    # xgboost.cv()交叉验证
    if useTrainCV:
        # 获取XGBClassifier的参数
        xgb_param = alg.get_xgb_params()
        
        # 训练集和测试集转换为xgboost
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        # xgtest = xgb.DMatrix(dtest[predictors].values)
        
        # xgboost.cv()交叉验证
        cvresult = xgb.cv(xgb_param,
                          xgtrain,
                          num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds,
                          metrics='logloss',
                          early_stopping_rounds=early_stopping_rounds, 
                          show_stdv=False)
        
        # 设置XGBClassifier参数
        alg.set_params(n_estimators=cvresult.shape[0])
    
    # 训练XGBClassifier
    alg.fit(dtrain[predictors],dtrain[target],eval_metric='logloss')
        
    # 模型预测
    dtrain_predictions = alg.predict(dtrain[predictors])         # 返回类别
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1] # 返回概率
        
    # 性能评估
    print("\nModel Report")
    print("logloss : %.6g" % metrics.log_loss(dtrain[target].values, dtrain_predictions))  # 负对数似然损失
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob)) # AUC值
    
    # 知道测试集target时使用
    #dtest['predprob'] = alg.predict_proba(dtest[predictors])[:,1]
    #results = test_results.merge(dtest[[IDcol,'predprob']], on='ID')
    #print('AUC Score (Test): %f' % metrics.roc_auc_score(results[target], results['predprob']))
    
    # 
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')


### 建立模型
print('### 建立XGBClassifier')
xgbc = XGBClassifier(learning_rate =0.1,                 # 学习率
                     silent = 1,                         # 输出中间过程
                     n_estimators=150,                   # 决策树个数
                     max_depth=5,                        # 决策树深度
                     min_child_weight=1,                 # 最小叶子节点权重和？
                     gamma=0,                            # 惩罚项系数
                     subsample=0.8,                      # 训练一棵树所用的数据占全部数据集比例
                     colsample_bytree=0.8,               # 训练一颗树所用的特征占全部数据集比例
                     objective= 'binary:logistic',       # 损失函数 
                     nthread=4,                          # 线程数
                     scale_pos_weight=1,                 # 样本不平衡
                     eval_metric='logloss',              # 评估指标
                     reg_alpha=0.03,                     # 正则化系数
                     seed=27)                            # 随机种子

### 网格搜索
## step1:决策树个数 n_estimators
print("### 调参:决策树个数")
#modelfit(xgbc, train, test, predictors)

## step2:决策树参数 max_depth/min_child_weight/gamma/subsample/colsample_bytree
print("### 调参:决策树参数")
param_test1 = {'max_depth':list(range(3,10,2)),'min_child_weight':list(range(1,6,2))}
param_test2 = {'max_depth':[4,5,6],'min_child_weight':[4,5,6]}
param_test2b ={'min_child_weight':[6,8,10,12]}
param_test3 = {'gamma':[i/10.0 for i in range(0,5)]}
param_test4 = {'subsample':[i/10.0 for i in range(6,10)],'colsample_bytree':[i/10.0 for i in range(6,10)]}
param_test5 = {'subsample':[i/100.0 for i in range(75,90,5)],'colsample_bytree':[i/100.0 for i in range(75,90,5)]}

## step3:正则化参数 reg_alpha
print("### 调参：正则化参数")
param_test6 = {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]}
param_test7 = {'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]}

gsearch = GridSearchCV(estimator = xgbc,         # 待调参模型
                       param_grid = param_test1, # 参数字典
                       scoring='neg_log_loss',   # 评价函数
                       n_jobs=-1,                 # 并行CPU个数
                       iid=False,                # 独立同分布
                       verbose=2,                # 显示中间过程
                       cv=5)                     # 5折交叉验证
                       
gsearch.fit(x_train,y_train)     # 训练GridSearchCV

print(gsearch.best_params_)                      # GridSearchCV最佳参数
print(gsearch.best_score_)                       # GridSearchCV最佳分数

xgbc = gsearch.best_estimator_                   # GridSEarchCV最佳分类器

### 训练XGBClassifier模型
print("### 训练XGBClassifier")
modelfit(xgbc, train, test, predictors)

### 建立XGBClassifier
### 调参:决策树个数
### 调参:决策树参数
### 调参：正则化参数
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
